In [3]:
#CTE de mejores vendedores, vendedores que mas generan ingresos
from src.db import run_sql
query= """WITH vendedor_top as (
		SELECT          SalesPersonID,
                        SUM(TotalPrice) as Ingreso_Generado
                        FROM sales
                        GROUP BY SalesPersonID
                        )
	 SELECT 
			vt.SalesPersonID,
            vt.Ingreso_Generado,
            e.FirstName,
            e.LastName
	 FROM vendedor_top AS vt
     JOIN employees as e
     ON vt.SalesPersonID = e.EmployeeID
     ORDER BY Ingreso_Generado DESC
     LIMIT 5;"""

df_customers = run_sql(query)
df_customers 

,SalesPersonID,Ingreso_Generado,FirstName,LastName
0,23,665091.0,Janet,Flowers
1,22,631158.0,Tonia,Mc Millan
2,21,618744.0,Devon,Brewer
3,20,569220.0,Shelby,Riddle
4,19,538954.0,Bernard,Moody


In [ ]:
#VISTA QUE INDICA EL INGRESO POR MES
from src.db import run_sql
query= """ 
SELECT * FROM ventas_total_mes
ORDER BY Mes ASC;"""

df_customers = run_sql(query)
df_customers 

,Año,Mes,TotalMensual
0,2025,1,708878.0
1,2025,2,633065.0
2,2025,3,742401.0
3,2025,4,692481.0
4,2025,5,710675.0
5,2025,6,729443.0
6,2025,7,728310.0
7,2025,8,738776.0
8,2025,9,718757.0
9,2025,10,731963.0


Función de ventana que utiliza ROW_NUMBER(), donde mostrara la posición del ranking en la que se encuentra el mes según el total vendido, donde se ve que el mes 3(marzo) se encuentra en el TOP 1 y el mes de Dicimebre, el mes donde el ingreso fue el más bajo del año. Además se agrego la función LAG() para comparar las ventas con el mes anterior y como extra se añadio el crecimiento (o caida) con respecto al mes anterior

In [1]:
from src.db import run_sql

query = """
SELECT
  Año,
  Mes,
  TotalMensual,

  -- 1) Ventas del mes anterior dentro del mismo año
  LAG(TotalMensual) 
    OVER (
      PARTITION BY Año 
      ORDER BY Mes
    ) AS TotalAnterior,

  -- 2) Cálculo del % de crecimiento respecto al mes anterior
  CASE 
    WHEN LAG(TotalMensual) OVER (PARTITION BY Año ORDER BY Mes) IS NULL 
      THEN NULL
    ELSE
      ROUND(
        ( TotalMensual 
          - LAG(TotalMensual) 
              OVER (PARTITION BY Año ORDER BY Mes)
        )
        / LAG(TotalMensual) 
            OVER (PARTITION BY Año ORDER BY Mes)
        * 100
      , 2)
  END AS CrecimientoPct,

  -- 3) Ranking global por TotalMensual (de mayor a menor)
  ROW_NUMBER() 
    OVER (
      ORDER BY TotalMensual DESC
    ) AS RowNum

FROM ventas_total_mes

-- orden final del resultado (aquí mostramos primero por año/mes,
-- pero el RowNum ya refleja el ranking por TotalMensual)
ORDER BY Año, Mes;
"""
df = run_sql(query)
df


,Año,Mes,TotalMensual,TotalAnterior,CrecimientoPct,RowNum
0,2025,1,708878.0,NaN,NaN,8
1,2025,2,633065.0,708878.0,-10.69,11
2,2025,3,742401.0,633065.0,17.27,1
3,2025,4,692481.0,742401.0,-6.72,10
4,2025,5,710675.0,692481.0,2.63,7
5,2025,6,729443.0,710675.0,2.64,4
6,2025,7,728310.0,729443.0,-0.16,5
7,2025,8,738776.0,728310.0,1.44,2
8,2025,9,718757.0,738776.0,-2.71,6
9,2025,10,731963.0,718757.0,1.84,3


Esta función ejecuta el procedimiento almacenado llamado "registro_productos", el cual funciona para almacenar de manera simple nuevos productos para vender

In [1]:
from src.db import execute_sql

query = """
CALL registro_productos(
    9998,
    'Chocolinas',
    200,
    5,
    'Low',
    '2024-06-02',
    'Durable',
    'Unknown',
    200
);
"""
execute_sql(query)


Probamos que funciona el procedimiento almacenado llamando a la tabla productos con el nuevo producto "Chocolinas"

In [2]:
from src.db import run_sql
query= """select * from products
where ProductName='Chocolinas';"""
df_customers = run_sql(query)
df_customers 

,ProductID,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays
0,9998,Chocolinas,200.0,5,Low,0 days,Durable,Unknown,200.0


Trigger de seguridad el cual muestra si el calculo realizado de Total Price se encuentra mal realizado, lo que evita errores o cargas fraudulentas. Se puede crear una tabla para almacenar estos registros erroneos o fraudulentos. Donde en el caso de que este mal la función muestra un error que dice 'Error: TotalPrice no coincide con Quantity * Precio - Discount.'

Nota: el trigger creado se encuentra en la carpeta sql


In [5]:
#Vamos a probarlo (caso de que se esta insertando una fila de Sales con el calculo del Total Price)
from src.db import execute_sql

query = """INSERT INTO sales (
    SalesID, 
    SalesPersonID, 
    CustomerID, 
    ProductID, 
    Quantity, 
    Discount, 
    TotalPrice,            
    SalesDate, 
    TransactionNumber
)
VALUES (
    99999, 
    1, 
    33133, 
    12, 
    2, 
    0.1,
    21.6,                
    '2025-03-15 12:00:00', 
    'TX-TEST-0001'
);"""

Se muestra el Indice creado, en la tabla Sales especificamente en la columna SaleaDate.

In [ ]:
from src.db import run_sql

# 1. “SHOW INDEX” recupera todos los índices de la tabla `sales`
df_indexes = run_sql("SHOW INDEX FRgitOM sales;")
df_indexes


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,sales,0,PRIMARY,1,SalesID,A,50000,None,None,,BTREE,,,YES,None
1,sales,1,idx_sales_salesperson,1,SalesPersonID,A,22,None,None,,BTREE,,,YES,None
2,sales,1,idx_sales_customer,1,CustomerID,A,38896,None,None,,BTREE,,,YES,None
3,sales,1,idx_sales_product,1,ProductID,A,448,None,None,,BTREE,,,YES,None
4,sales,1,idx_sales_salesdate,1,SalesDate,A,49432,None,None,YES,BTREE,,,YES,None
